# Lecture #14: Neural Network Models for Regression
## AM 207: Advanced Scientific Computing
### Stochastic Methods for Data Analysis, Inference and Optimization
### Fall, 2019

<img src="fig/logos.jpg" style="height:150px;">

In [2]:
### Import basic libraries
from autograd import numpy as np
from autograd import grad
import numpy
import scipy as sp
import pandas as pd
import sklearn as sk
from sklearn.datasets.samples_generator import make_blobs
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML
from IPython.display import YouTubeVideo
%matplotlib inline

## Administrative Matters

1. **Attendance Quiz:**<br><br>
2. **Projects:** Read Check Point #2 carefully -- **you** are responsible for setting up meetings with your TF and instructors

## Outline
1. Regression as Generalized Linear Models
2. Neural Networks
3. Automatic Differentiation and BackPropagation

# Regression as Generalized Linear Models

## Linear Regression Models
Here is a generalized linear model (GLM) you've known since day one!
\begin{align}
\mu &= \mathbf{w}^\top \mathbf{X}^{(n)}\\
Y^{(n)}&\sim \mathcal{N}(\mu, \sigma^2)
\end{align}
Alternatively, we can write this model as

\begin{align}
    Y^{(n)} = \mathbf{w}^\top \mathbf{X}^{(n)} + \epsilon; \quad \epsilon \sim \mathcal{N}(\mu, \sigma^2)
\end{align}

That is, injecting covariates into a normal likelihood is precisely linear regression! Just like in the case of logistic regression, we can form scientific hypotheses by examining the parameters of a linear regression model:

\begin{align}
    \widehat{\text{income}} = 2 * \text{education (yr)} + 3.1 * \text{married} - 1.5 * \text{gaps in work history}
\end{align}

## How Would You Parameterize a Non-linear Trend?
<img src="./fig/fig12.png" style='height:400px;'>
It's not easy to think of a function $g(x)$ can capture the trend in the data, e.g. what degree of polynomial should we use?

## Review of the Geometry of Logistic Regression 
In **logistic regression**, we model the probability of an input $\mathbf{x}$ being labeled '1' as a function of its distance from the hyperplane parametrized by $\mathbf{w}$
<img src="./fig/fig0.png" style='height:300px;'>
That is, we model $p(y=1 | \mathbf{w}, \mathbf{x}) = \sigma(\mathbf{w}^\top \mathbf{x})$. Where $\mathbf{w}^\top \mathbf{x}=0$ is the equation of the decision boundary.

## How would you parametrize a ellipitical decision boundary?

<img src="./fig/fig1.png" style='height:300px;'>

We can say that the decision boundary is given by a ***quadratic function*** of the input:
$$
w_1x^2_1 + w_2x^2_2 + w_3 = 0
$$
We say that we can fit such a decision boundary using logistic regression with degree 2 polynomial features

## How would you parametrize an arbitrary complex decision boundary?

<img src="./fig/fig2.png" style='height:300px;'>

It's not easy to think of a function $g(x)$ can capture this decision boundary.

**GOAL:** Find models that can capture *arbitrarily complex* functions.

# Neural Networks

## Approximating Arbitrarily Complex Decision Boundaries

Given an exact parametrization, we could learn the functional form, $g$, of the decision boundary directly. 

However, assuming an exact form for $g$ is restrictive. 

Rather, we can build increasingly good approximations, $\widehat{g}$, of $g$ by composing simple functions. 

## What is a Neural Network?

**Goal:** build a good approximation $\widehat{g}$ of a complex function $g$ by composing simple functions.

For example, let the following picture represents $a = f\left(\sum_{i}w_ix_i\right)$, where $f$ is a non-linear transform, and we denote the intermediate value $\sum_{i}w_ix_i$ by $s$.

<img src="./fig/fig4.png" style='height:300px;'>

**Note:** we always assume that $x_0=1$ and hence $w_0$ is the intercept or ***bias*** of the linear expression $\sum_i w_i x_i$.

## Exercise: Translate Graphical Representations into Functional Ones

Translate the following graphical representation of a function into a functional form, i.e. $g(x) = ?$ 
<img src="./fig/fig7.png" style='height:150px;'>
Use the following notation:

**1.** the input node is $x$, the layer is 0<br>
**2.** the weight from the $i$-th node in the $l$-th layer to the $j$-th node in the $l+1$-th layer is $w^l_{ij}$<br>
**3.** the activation function $f(z) = e^{-0.5z^2}$

## Neural Networks as Function Approximators

Then we can define the approximation $\widehat{g}$ with a graphical schema representing a complex series of compositions and sums of the form, $f\left(\sum_{i}w_ix_i\right)$

<img src="./fig/fig5.png" style='height:300px;'>

This is a ***neural network***. We denote the weights of the neural network collectively by $\mathbf{W}$.
The non-linear function $f$ is called the ***activation function***.

**Note:** 
Typically, at each node, we want to take a weighted sum of the values of the previous nodes with a additional ***bias term***. That is, we want as input $\sum_i w^l_{ij} \text{node}^l_i + \text{bias}_j$ for the $j$-th node in the $l$-th hidden layer. This is often done by adding an extra node per layer with the value of 1:
<img src="./fig/bias.png" style='height:250px;'>
The bias terms are considered to be part of the network parameters and when we jointly denote the network parameters by $\mathbf{W}$, bias terms are included.

## A Flexible Framework for Function Approximation


<img src="./fig/fig6.png" style='height:500px;'>

## Common Choices for the Activation Function

<img src="./fig/fig8.png" style='height:500px;'>

## Neural Networks are Universal Function Approximators

So what kind of functions can be approximated by neural networks?

**Theorem: (Hornik, Stinchombe, White, 1989)** Fix a "nice" activation function $f$. For any continuous function $g$ on a compact set $K$, there exists a feedforward neural network with activation $f$, having only a single hidden layer, which approximates $g$ to within an arbitrary degree of precision on $K$.

For this reason, we call neural networks ***universal function approximators***.

## Neural Networks Regression

**Model for Regression:** $Y^{(n)}\sim \mathcal{N}(\mu, \sigma^2)$, $\mu = g_\mathbf{W}(\mathbf{X}^{(n)})$, where $g_\mathbf{W}$ is a neural network with parameters $\mathbf{W}$.

**Training Objective:** find $\mathbf{W}$ to maximize the likelihood of our data. This is equivalent to minimizing the Mean Square Error,
$$
\max_{\mathbf{W}}\, \mathrm{MSE}(\mathbf{W}) = \frac{1}{N}\sum^N_{n=1} \left(y_n - g_\mathbf{W}(x_n)\right)^2
$$

**Optimizing the Training Objective:** For linear regression (when $g_\mathbf{W}$ is a linear function), we computed the gradient of the MSE with respective to the model parameters $\mathbf{W}$, set it equal to zero and solved for the optimal $\mathbf{W}$ analytically (see Homework #0). For logistic regression, we computed the gradient and used (stochastic) gradient descent to "solve for where the gradient is zero".

Can we do the same when $g_\mathbf{W}$ is a neural network?

# Automatic Differentiation and Backpropagation

## Gradient Computation for Neural Networks

Computing the gradient for any parameter $w^l_{ij}$ in the following network requires us to use the ***chain rule***:

\begin{align}
\frac{\partial}{\partial t} g(h(t)) = g'(h(t))h'(t),\quad& \text{or}\quad\frac{\partial g}{\partial t} = \frac{\partial g}{\partial h} \frac{\partial h}{\partial t}
\end{align}

This is because a neural network is just a big composition of functions.

<img src="./fig/fig7.png" style='height:150px;'>

## Example: Computing Neural Network Gradients

<img src="./fig/backprop.png" style='height:600px;'>

## Backpropagation: Gradient Descent for Neural Networks

The ***backpropagation*** algorithm consists of three phases:
0. (**Initialize**) intialize the network parameters $\mathbf{W}$
1. Repeat:
  1. (**Forward Pass**) compute all intermediate values $s_{ij}^l$ and $a_{ij}^l$ for the given covariates $\mathbf{X}$
  2. (**Backward Pass**) compute all the gradients $\frac{\partial \mathcal{L}}{\partial w^l_{ij}}$
  3. (**Update Parameters**) update each parameter by $-\eta \frac{\partial \mathcal{L}}{\partial w^l_{ij}}$
  
<img src="./fig/graph_structure.png" style='height:200px;'>

## Gradient Computation with Automatic Differentiation

The forwards-backwards way of computing the gradient lends itself to an algorithm that automates gradient computation for any neural network. 

This is a special instance of ***reverse mode automatic differentiation*** -- a method of algorithmically computing exact gradients for functions defined by combinations of simple functions, by drawing graphical models of the composition of functions and then taking gradients by going forwards-backwards.
<img src="./fig/function.png" style='height:50px;'>

<img src="./fig/computation_graph.png" style='height:150px;'>


# With Great Flexibility Comes with Great Problems

## Neural Network Regression vs Linear Regression

Linear models are easy to interpret. Once we've found the MLE of the model parameters, we can formulate scientific hypotheses about the relationship between the outcome $Y$ and the covariates $\mathbf{X}$:

\begin{align}
    \widehat{\text{income}} = 2 * \text{education (yr)} + 3.1 * \text{married} - 1.5 * \text{gaps in work history}
\end{align}

What do the weights of a neural network tell you about the relationship between the covariates and the outcome?
<img src="./fig/fig5.png" style='height:250px;'>

## Generalization Error and Bias/Variance
Complex models have ***low bias*** -- they can model a wide range of functions, given enough samples.

But complex models like neural networks can use their 'extra' capacity to explain non-meaningful features of the training data that are unlikely to appear in the test data (i.e. noise). These models have ***high variance*** -- they are very sensitive to small changes in the data distribution, leading to drastic performance decrease from train to test settings.

<table>
    <tr>
        <td>
            <img src="./fig/fig11.png" style="width: 380px;" align="center"/>
        </td>
        <td>
            <img src="./fig/fig12.png" style="width: 380px;" align="center"/>
        </td>
    </tr>
</table>